The 'tutorial' example, see:

https://magmasource.caltech.edu/forum/index.php?board=12.0

https://magmasource.caltech.edu/forum/index.php/board=31.0

Converted to ipynb using https://jupytext.readthedocs.io/en/latest/install.html

In [ ]:
%%capture
!apt-get update
!apt-get install gsl-bin

In [ ]:
%%capture
!pip3 install tinynumpy

In [ ]:
import os
data_directory = "/content/MY_DATA/"

# Create the directory if it doesn't exist
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

# Change the current working directory
os.chdir(data_directory)

In [ ]:
# Need to upload: libalphamelts.so, meltsdynamic.py, meltsengine.py, meltsstatus.py

from google.colab import files
# Upload the files
uploaded_files = files.upload()

In [ ]:
# Check that they uploaded correctly
!ls

In [ ]:
# Can use to connect an interactive console to notebook
%connect_info

In [ ]:
from meltsdynamic import MELTSdynamic

In [ ]:
try:
    import numpy as np
    import pandas as pd
except:
    from tinynumpy import tinynumpy as tnp
    np = None
    pd = None

In [ ]:
try:
    import matplotlib.pyplot as plt
    %matplotlib inline
except:
    plt = None

In [ ]:
liquidus = MELTSdynamic(1)

In [ ]:
pressure = 500.0
temperature = 1200.0

In [ ]:
bulk = [48.68, 1.01, 17.64, 0.89, 0.03, 7.59, 0.0, 9.10, 0.0, 0.0, 12.45, 2.65, 0.03, 0.08, 0.2, 0.0, 0.0, 0.0, 0.0]

In [ ]:
liquidus.engine.setBulkComposition(bulk)
liquidus.engine.pressure = pressure

In [ ]:
# Note that as of Nov 18th, 2020 temperature is in Celcius
liquidus.engine.temperature = temperature

In [ ]:
%%capture output --no-stdout --no-display
# This should capture all output but a little was going to stdout, which was a bug that has since been fixed
liquidus.engine.findLiquidus()
temperature = liquidus.engine.temperature

In [ ]:
print(f'temperature = {temperature:.2f}')

In [ ]:
liquidusPhase = liquidus.engine.solidNames
print(liquidusPhase)

In [ ]:
# Note that as of July 4th, 2022 .calcSaturationState() outputs wt% oxides
liquidus.engine.calcEndMemberProperties(liquidusPhase, liquidus.engine.dispComposition[liquidusPhase[0]])
Xan = liquidus.engine.getProperty('X', liquidusPhase, "CaAl2Si2O8")
print(Xan)

In [ ]:
liquidus.engine.setSystemProperties("Mode", "Fractionate Solids")

In [ ]:
# .addNodeAfter() uses .copy(), which only copies "INPUT" or "INPUT AND OUTPUT"
# .copyAndKeepOutput() copies "OUTPUT" also. We can verify that "OUTPUT" is reset
# in ptpath when .calcEquilibriumState() (or .findLiquidus()) is called.
ptpath = liquidus.copyAndKeepOutput()

In [ ]:
%%capture output --no-stdout --no-display
# Run mode is 1 (isobaric, isothermal) if not defined
# Select output mode 0, to equilibrate but not do any fractionations
ptpath.engine.calcEquilibriumState(1, 0)
print(ptpath.engine.status.message)
print(ptpath.engine.solidNames)

In [ ]:
%%capture output --no-display
while ptpath.engine.temperature >= 1000:
    ptpath = ptpath.addNodeAfter()
    ptpath.engine.temperature = ptpath.engine.temperature - 3

    # Run mode is saved from previous run
    # Select 1 to get output after equilibration and before fractionation
    ptpath.engine.calcEquilibriumState(1, 1)
    print(ptpath.engine.status.message)

    # Can populate X, mu0 ,mu as go along (should test that the phase exists)
    # This is a Supplemental Calculator-type method so "OUTPUT" is not reset
    if "plagioclase1" in ptpath.engine.solidNames:
        ptpath.engine.calcEndMemberProperties("plagioclase1", ptpath.engine.dispComposition["plagioclase1"])

In [ ]:
# Need to upload: run-alphamelts.command, column-pick.command

from google.colab import files
# Upload the files
uploaded_files = files.upload()

In [ ]:
!chmod +x *.command
!ls -l *.command

In [ ]:
# Get the Phase_mass_tbl.txt and Phase_vol_tbl.txt files if standalone alphaMELTS 2 is installed
# On CoLab we don't do a full install - just upload the two scripts that manipulate output
!./run-alphamelts.command -x

In [ ]:
temp = ptpath.getListProperty('temperature')
mgo = ptpath.getListProperty('dispComposition', 'liquid1', 'MgO')
mass = ptpath.getListProperty('mass', 'bulk')

In [ ]:
# A silly example to illustrate sorting the list (in this case by temperature ascending)
# There are multiple ways to do this.
if np is not None:
    indices = np.arange(ptpath.Last.nodeIndex+1)
else:
    indices = list(range(ptpath.Last.nodeIndex+1))

In [ ]:
if plt is not None:

    plt.scatter(indices, mgo, marker='*', color='black')

In [ ]:
if np is not None:
    temp = np.array(temp)
    order = indices[temp.argsort()]
else:
    order = [x for _, x in sorted(zip(temp, indices), key=lambda pair: pair[0])]

In [ ]:
ptpath.sortListBy(order)
mgo = ptpath.getListProperty('dispComposition', 'liquid1', 'MgO')

In [ ]:
if plt is not None:
    plt.scatter(indices, mgo, marker='*', color='blue')
    plt.show()

In [ ]:
# flip it back again
ptpath.sortListBy(order)
mgo = ptpath.getListProperty('dispComposition', 'liquid1', 'MgO')

In [ ]:
Xan = ptpath.getListProperty('X', "plagioclase1", "CaAl2Si2O8")

In [ ]:
# If we didn't calculate Xan as we went along and wanted to avoid repeating the equilibration loop.
plag = ptpath.getListProperty('dispComposition', 'plagioclase1')

In [ ]:
# Need to give the feldspars from different runs different names that start with feldspar
phaseList = ['plagioclase_'] * (ptpath.Last.nodeIndex+1)
for i in range(ptpath.Last.nodeIndex+1):
    phaseList[i] += str(i+1)

In [ ]:
# A pandas example
if pd is not None:
    table = pd.DataFrame(data=np.asarray(plag), index=ptpath.endMemberFormulas["bulk"], columns=phaseList)
    print(table)

In [ ]:
# This works because Xan is independent of pressure and temperature but we need to specify them
# start a new list (as ptpath already has a value for Xan)
melts = MELTSdynamic(1)
melts.engine.temperature = ptpath.engine.temperature
melts.engine.pressure = ptpath.engine.pressure

There is no feldspar in the first ptpath phase assemblage (composition will be all zeroes; check SiO2)
there are other ways to do this (see meltsplotter.py); .tolist() on phaseList is optional.
indices = np.isfinite(plag[0, :].astype(np.double))
phaseList = (np.array(phaseList)[indices])
melts.engine.calcEndMemberProperties(phaseList, plag[:, indices])

In [ ]:
# There is feldspar in the first ptpath now, as alphaMELTS for Python always uses findWetLiquidus
melts.engine.calcEndMemberProperties(phaseList, plag)

In [ ]:
#To get properties from a single node, use getNodeProperty with None for the index.
#Or equivalently use the engine's getProperty method instead i.e:
#Xan = melts.engine.getProperty('X', phaseList[indices], "CaAl2Si2O8")

In [ ]:
Xan = melts.getNodeProperty(None, 'X', phaseList, "CaAl2Si2O8")

In [ ]:
# Need to upload: meltsplotter.py, tasplot.py

from google.colab import files
# Upload the files
uploaded_files = files.upload()

In [ ]:
# Ryan Clark's MELTSplotter, using matplotlib
if plt is not None:
    try:
        from meltsplotter import MELTSplotter
        mplot1 = MELTSplotter()
        mplot1 = mplot1.plotTAS(ptpath)
        mplot2 = MELTSplotter()
        mplot2 = mplot2.harkerPlot(ptpath, 'SiO2', ['CaO', 'MgO', 'K2O', 'TiO2', 'Na2O', 'FeO', 'MnO', 'Al2O3'], False)
        mplot3 = MELTSplotter()
        mplot3 = mplot3.harkerPlot(ptpath, 'MgO', ['SiO2', 'CaO', 'K2O', 'TiO2', 'Na2O', 'FeO', 'MnO', 'Al2O3'], False)
    except:
        print("Could not find MELTSplotter!")

In [ ]:
temp = ptpath.getListProperty('temperature')
sio2 = ptpath.getListProperty('dispComposition', 'liquid1', 'SiO2')
feo = ptpath.getListProperty('dispComposition', 'liquid1', 'FeO')
#mgo = ptpath.getListProperty('dispComposition', 'liquid1', 'MgO')
cao = ptpath.getListProperty('dispComposition', 'liquid1', 'CaO')
na2o = ptpath.getListProperty('dispComposition', 'liquid1', 'Na2O')

In [ ]:
%%capture
!apt-get install gmt
!apt-get install -y ghostscript

In [ ]:
%%capture
!pip3 install pygmt
!ln -s /lib/x86_64-linux-gnu/libgmt.so.6 /lib/x86_64-linux-gnu/libgmt.so

In [ ]:
# Need to upload: legend.txt

from google.colab import files
# Upload the files
uploaded_files = files.upload()

In [ ]:
try:
    import pygmt
    #pygmt.show_versions()
    fig = pygmt.Figure()
    fig.basemap(region=[1000, 1220, 0, 100], projection="X-4i/3i", frame=['WeSn+tMelt composition', 'x+lT @.C system', 'y+Lwt%'])
    fig.plot(x=temp, y=sio2, pen="thin,lightblue", style="c0.1")
    fig.plot(x=temp, y=feo, pen="thin,orange", style="c0.1")
    fig.plot(x=temp, y=mgo, pen="thin,magenta", style="c0.1")
    fig.plot(x=temp, y=cao, pen="thin,gray", style="c0.1")
    fig.plot(x=temp, y=na2o, pen="thin,pink", style="c0.1")
    try:
        fig.legend(position='jTL', box=True, spec='legend.txt')
    except:
        print("Could not find legend.txt")
    fig.show()
except:
    print("Could not find pygmt")